In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [ ]:
import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from IPython import display

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
from tensorflow.keras.utils import plot_model

import warnings
warnings.filterwarnings("ignore")

# Load Data

In [ ]:
root_dir = "/kaggle/input/simpsons-faces/cropped"

In [ ]:
image_paths = []
for file in os.listdir(root_dir):
    image_paths.append(os.path.join(root_dir, file))

In [ ]:
def load_files(image_paths):
    images = []
    for file in tqdm(image_paths):
        image = Image.open(file)
        image = image.resize((64, 64))
        image = (np.array(image).astype('float32') - 127.5) / 127.5
        images.append(image)

    return np.array(images)

In [ ]:
train_images = load_files(image_paths)

# Visualization

In [ ]:
def visualize_df(df: pd.DataFrame):
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))

    for i, ax in enumerate(axes.ravel()):
        if i < len(df):
            a = np.random.randint(1, len(df), 1)[0]
            img_path = df.loc[a][['image_path']].values[0]
            
            image = Image.open(img_path).convert('RGB')
            
            ax.imshow(image)
            ax.set_title(f"File: {img_path.split('/')[-1]}")
            ax.axis('off')
            
        else:
            ax.axis('off')
            
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_df(pd.DataFrame({"image_path": image_paths}))

# Model

In [ ]:
g_noise_input = layers.Input(shape=(100,))

g_hidden1 = layers.Dense(8 * 8 * 128)(g_noise_input)
g_reshaped_input = layers.Reshape((8, 8, 128))(g_hidden1)

g_upsample1 = layers.Conv2DTranspose(filters=128, kernel_size=(4, 4), strides=(2, 2), padding='same')(g_reshaped_input)
g_relu1 = layers.ReLU()(g_upsample1)

g_upsample2 = layers.Conv2DTranspose(filters=256, kernel_size=(4, 4), strides=(2, 2), padding='same')(g_relu1)
g_relu2 = layers.ReLU()(g_upsample2)

g_upsample3 = layers.Conv2DTranspose(filters=512, kernel_size=(4, 4), strides=(2, 2), padding='same')(g_relu2)
g_relu3 = layers.ReLU()(g_upsample3)

g_output = layers.Conv2D(filters=3, kernel_size=(5, 5), activation='tanh', padding='same')(g_relu3)

generator_model = models.Model(inputs=g_noise_input, outputs=g_output)

In [ ]:
plot_model(generator_model)

In [ ]:
d_input_layer = layers.Input(shape=(64, 64, 3))

d_conv1 = layers.Conv2D(filters=64, kernel_size=(4, 4), strides=(2, 2), padding='same')(d_input_layer)
d_leaky_relu1 = layers.LeakyReLU(alpha=0.2)(d_conv1)

d_conv2 = layers.Conv2D(filters=128, kernel_size=(4, 4), strides=(2, 2), padding='same')(d_leaky_relu1)
d_leaky_relu2 = layers.LeakyReLU(alpha=0.2)(d_conv2)

d_flatten = layers.Flatten()(d_leaky_relu2)

d_dropout = layers.Dropout(0.3)(d_flatten)

d_output_layer = layers.Dense(1, activation='sigmoid')(d_dropout)

discriminator_model = models.Model(inputs=d_input_layer, outputs=d_output_layer)

In [ ]:
plot_model(discriminator_model)

# Train

In [ ]:
EPOCHS = 30
NUM_TESTS = 16
NOISE_DIM = 100
BUFFER_SIZE = len(train_images)
BATCH_SIZE = 128
LR_G = 0.0004
LR_D = 0.0004
BETA_G = 0.5
BETA_D = 0.5

generator_optimizer = optimizers.Adam(learning_rate=LR_G, beta_1=BETA_G)
discriminator_optimizer = optimizers.Adam(learning_rate=LR_D, beta_1=BETA_D)
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
def generator_loss(fake_output):
	return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def show_generated_images(epoch, generator, test_images, output):
    generated_images = generator(test_images, training=False)
    
    fig = plt.figure(figsize=(10, 10))
    for i in range(generated_images.shape[0]):
        plt.subplot(4, 4, i+1)
        img = (generated_images[i, :, :, :] + 1.0) * 127.5
        plt.tight_layout()
        plt.imshow(np.array(img).astype('uint8'))
        plt.axis("off")
        
    if not os.path.exists(output):
        os.makedirs(output)

    fig.savefig(f"{output}/dcgan_{epoch}.png")
    plt.show()

In [ ]:
def train_step(images, generator, discriminator, cross_entropy, generator_optimizer, discriminator_optimizer):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])
   
    with tf.GradientTape() as disc_tape:
        fake_images = generator(noise, training=True)
        real_preds = discriminator(images, training=True)
        fake_preds = discriminator(fake_images, training=True)
        disc_loss = discriminator_loss(real_preds, fake_preds)
        
    discriminator_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))  
    
    with tf.GradientTape() as gen_tape:
        fake_images = generator(noise, training=True)
        fake_preds = discriminator(fake_images, training=True)
        gen_loss = generator_loss(fake_preds)
        
    generator_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
    return gen_loss, disc_loss

In [ ]:
test_images = tf.random.normal([NUM_TESTS, NOISE_DIM])
g_loss = []
d_loss = []

for epoch in range(EPOCHS):
    for image_batch in dataset:
        gen_loss, disc_loss = train_step(image_batch, generator_model, discriminator_model, cross_entropy, generator_optimizer, discriminator_optimizer)
        
    g_loss.append(gen_loss)
    d_loss.append(disc_loss)
    print(f"[+] Epoch {epoch + 1}, Generator Loss: {gen_loss}, Discriminator Loss: {disc_loss}")
    show_generated_images(epoch, generator_model, test_images, "output")
    display.clear_output(wait=True)

# GIF

In [ ]:
!rm "output/dcgan_simpsons.gif"

In [ ]:
import imageio
import glob

files = glob.glob("./output/*.png")
gif_file = "./output/dcgan_simpsons.gif"

with imageio.get_writer(gif_file, mode="I") as writer:
    for i, file in enumerate(files):
        frame = 2 * (i)
        if round(frame) > round(last):
            last = frame
        else:
            continue
            
        img = imageio.imread(file)
        writer.append_data(img)
        
    img = imageio.imread(file)
    writer.append_data(img)

In [ ]:
display.Image(filename=gif_file)

![](output/dcgan_simpsons.gif)